In [62]:
import numpy as np

# create samples
samples = []
filename = '../data/obj_pose-laser-radar-synthetic-input.txt'
readfile = open(filename,'r')


In [65]:
x = np.array([0., 0., 0., 0.])

x_all = np.empty((0,4))
x_all = np.append(x_all, [x], axis=0)

P = np.array([[1, 0, 0, 0],
             [0, 1, 0, 0],
             [0, 0, 100, 0],
             [0, 0, 0, 100]], dtype = float)

p00 = np.array([P[0, 0]])
p11 = np.array([P[1, 1]])
p22 = np.array([P[2, 2]])
p33 = np.array([P[3, 3]])

R_lidar = np.array([[0.0225, 0], [0, 0.0225]])
R_radar = np.array([[0.09, 0, 0], [0, 0.0009, 0], [0, 0, 0.09]])

H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], dtype = float) # H_lidar

In [75]:
# Calculate Jacobian

def CalculateJacobian(x):

    H = np.zeros((3,4), dtype=float) # H_radar

    px = float(x[0])
    py = float(x[1])
    vx = float(x[2])
    vy = float(x[3])

    den1 = float(np.sqrt(px*px + py*py))
    den2 = float(px*px + py*py)
    den3 = float(den1*den2)

    H[0,0] = px/den1
    H[0,1] = py/den1
    H[0,2] = 0.0
    H[0,3] = 0.0
    H[1,0] = -py/den2
    H[1,1] = px/den2
    H[1,2] = 0.0
    H[1,3] = 0.0
    H[2,0] = py*(vx*py - vy*px)/den3
    H[2,1] = px*(vy*px - vx*py)/den3
    H[2,2] = px/den1
    H[2,3] = py/den1

    return H

In [76]:
# Lidar measurement

k = 0
n = 2

is_init = False

timestamp = []
x_groundtruth = []
y_groundtruth = []
vx_groundtruth = []
vy_groundtruth = []

for k in range(n):
    line = readfile.readline()
    words = line.split('\t')
    
    if words[0] == 'L': 
        x_measured = float(words[1])
        y_measured = float(words[2])
        timestamp = np.append(timestamp, float(words[3]))
        x_groundtruth = np.append(x_groundtruth, float(words[4]))
        y_groundtruth = np.append(y_groundtruth, float(words[5]))
        vx_groundtruth = np.append(vx_groundtruth, float(words[6]))
        vy_groundtruth = np.append(vy_groundtruth, float(words[7]))
        
        z = np.array([x_measured, y_measured])
    
        t_current = timestamp[k]
        if is_init == False:
            t_previous = t_current
            is_init = True
            print(is_init)
            
        dt = t_current - t_previous        
        print('dt = ', str(dt))
        
        F = np.array(
            [[1, 0, dt, 0],
             [0, 1, 0, dt],
             [0, 0, 1, 0],
             [0, 0, 0, 1]])
        
        dt4 = dt*dt*dt*dt;
        dt3 = dt*dt*dt;
        dt2 = dt*dt;
        sx2 = 9;
        sy2 = 9;

        Q = [[dt4/4*sx2,      0,    dt3/2*sx2,  0],
             [0,    dt4/4*sy2,   0,     dt3/2*sy2],
             [dt3/2*sx2,      0,    dt2*sx2,    0],  
             [0,    dt3/2*sy2,   0,     dt2*sy2]]
        
        # Prediction
        x = np.matmul(F,x)
        P = np.matmul( np.matmul(F, P), F.T)  + Q
        
        # Update
        y = z - np.matmul(H, x)

        H_t = H.T
        S = np.matmul(np.matmul(H, P), H_t) + R_lidar
        S_i = np.linalg.inv(S)
        K =  np.matmul(np.matmul(P, H_t), S_i)

        x = x + np.matmul(K, y);
        P = np.matmul((np.eye(4) - np.matmul(K, H)), P)

        # store data
        x_all = np.append(x_all, [x], axis=0)
        p00 = np.append(p11, [P[0, 0]], axis = 0)
        p11 = np.append(p11, [P[1, 1]], axis = 0)
        p22 = np.append(p11, [P[2, 2]], axis = 0)
        p33 = np.append(p11, [P[3, 3]], axis = 0)

    if words[0] == 'R':
        radar_rho_measured = float(words[1])
        radar_phi_measured = float(words[2])
        radar_rhodot_measured = float(words[3])
        timestamp = np.append(timestamp, float(words[4]))
        x_groundtruth = np.append(x_groundtruth, float(words[5]))
        y_groundtruth = np.append(x_groundtruth, float(words[6]))
        vx_groundtruth = np.append(y_groundtruth, float(words[7]))
        vy_groundtruth = np.append(vy_groundtruth, float(words[8]))
        
        z = np.array([radar_rho_measured, radar_phi_measured, radar_rhodot_measured])
                        
        t_current = timestamp[k]
        if is_init == False:
            t_previous = t_current
            is_init = True
            print(is_init)
            
        dt = t_current - t_previous        
        print('dt = ', str(dt))
        
        F = np.array(
            [[1, 0, dt, 0],
             [0, 1, 0, dt],
             [0, 0, 1, 0],
             [0, 0, 0, 1]])
        
        dt4 = dt*dt*dt*dt;
        dt3 = dt*dt*dt;
        dt2 = dt*dt;
        sx2 = 9;
        sy2 = 9;

        Q = [[dt4/4*sx2,      0,    dt3/2*sx2,  0],
             [0,    dt4/4*sy2,   0,     dt3/2*sy2],
             [dt3/2*sx2,      0,    dt2*sx2,    0],  
             [0,    dt3/2*sy2,   0,     dt2*sy2]]
        
        # Prediction
        x = np.matmul(F,x)
        P = np.matmul( np.matmul(F, P), F.T)  + Q

        
        # Update
        Hj = np.zeros((3,4), dtype=float) 
        Hj = CalculateJacobian(x)
        
        px = float(x[0])
        py = float(x[1])
        vx = float(x[2])
        vy = float(x[3])
        
        rho = np.sqrt( px*px + py*py )
        if rho!= 0:
            hx = np.array([rho, np.arctan2( py, px ), ( px*vx + py*vy )/rho])
        else:
            hx = np.array([rho, np.arctan2( py, px ), 0])
            
        y = z - hx
          
        if ( y[1] > np.pi ):
            y[1] -= 2*np.pi
        if ( y[1] < -np.pi ):
            y[1] += 2*np.pi

        Hj_t = Hj.T
        
        print(np.shape(Hj))
        print(np.shape(P))
        print(np.shape(R_radar))
        
        S = np.matmul(np.matmul(Hj, P), Hj_t) + R_radar
        S_i = np.linalg.inv(S)
        K =  np.matmul(np.matmul(P, Hj_t), S_i)

        x = x + np.matmul(K, y);
        P = np.matmul((np.eye(4) - np.matmul(K, Hj)), P)

        # store data
        x_all = np.append(x_all, [x], axis=0)
        p00 = np.append(p11, [P[0, 0]], axis = 0)
        p11 = np.append(p11, [P[1, 1]], axis = 0)
        p22 = np.append(p11, [P[2, 2]], axis = 0)
        p33 = np.append(p11, [P[3, 3]], axis = 0)
        
           
print(x_all)
print(p00)

True
dt =  0.0
dt =  50000.0
(3, 4)
(4, 4)
(3, 3)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.05371834e-01  5.67569487e-01  0.00000000e+00  0.00000000e+00]
 [ 1.17384800e+00  4.81072900e-01  3.47390463e-05 -3.45986343e-06]
 [ 1.65062600e+00  6.24690400e-01 -1.56679259e-05  9.20456332e-06]
 [ 2.18882400e+00  6.48739200e-01  3.71958454e-05 -8.24261116e-06]
 [ 2.65525600e+00  6.65979800e-01 -1.85385649e-05  8.93223501e-06]
 [ 3.01222300e+00  6.37045500e-01  3.28172445e-05 -1.00896068e-05]
 [ 3.89365000e+00  3.11793000e-01  2.43983580e-06 -2.92049322e-06]
 [ 4.30934600e+00  5.78563700e-01  1.41880041e-05  1.35913211e-05]
 [ 4.65427106e+00  6.44818066e-01  4.96268519e+00  1.24406523e+00]]
[1.00000000e+00 2.20048900e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.56125116e+03 0.00000000e+00
 0.00000000e+00 7.13961473e+11]


In [77]:
# Radar measurement

line = readfile.readline()
words = line.split('\t')

if words[0] != 'R':
    print('Not radar data')
    
radar_rho_measured = words[1]
radar_phi_measured = words[2]
radar_rhodot_measured = words[3]
timestamp = words[4]
x_groundtruth = words[5]
y_groundtruth = words[6]
vx_groundtruth = words[7]
vy_groundtruth = words[8]

print(words)

Not radar data
['L', '4.351431e+00', '8.991741e-01', '1477010443800000', '4.754374e+00', '6.669207e-01', '5.176340e+00', '2.425866e-01', '4.683024e-02', '1.166039e-01\n']


In [ ]:
x = np.array([1, 2])
A = np.array([[1, 2],[3, 4]])
print(A*x[0])

In [ ]:
x[:,0]